In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Extracting features of 2018 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [4]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN


In [5]:
df.shape

(274, 7)

In [6]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'e012ce791f204b738862e05af4396cb3'

In [7]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [8]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [9]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN,Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN,Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],NaN,Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN,Drama Western Thriller
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN,Action Thriller Mystery


In [10]:
df_2018 = df[['Title','Cast and crew','genres']]

In [11]:
df_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama Western Thriller
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery
...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
270,Vice,Adam McKay (director/screenplay); Christian Ba...,Action Crime Thriller
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Documentary


In [12]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [13]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))

In [14]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))

In [15]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))

In [16]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])
    
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [17]:
df_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama Western Thriller,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
...,...,...,...,...,...,...,...
269,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall
270,Vice,Adam McKay (director/screenplay); Christian Ba...,Action Crime Thriller,Adam McKay,Christian Bale,Amy Adams,Steve Carell
271,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux
272,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Documentary,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell


In [18]:
df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [19]:
new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [20]:
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,The Strange Ones
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Stratton
3,Warwick Thornton,Bryan Brown,Sam Neill,NaN,Drama Western Thriller,Sweet Country
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,The Commuter
...,...,...,...,...,...,...
269,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime,Holmes & Watson
270,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Action Crime Thriller,Vice
271,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History,On the Basis of Sex
272,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Documentary,Destroyer


In [21]:
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')

new_df18['movie_title'] = new_df18['movie_title'].str.lower()

In [22]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']

In [23]:
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama Western Thriller,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
269,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Mystery Adventure Comedy Crime,holmes & watson,Will Ferrell John C. Reilly Rebecca Hall Etan ...
270,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Action Crime Thriller,vice,Christian Bale Amy Adams Steve Carell Adam McK...
271,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Drama History,on the basis of sex,Felicity Jones Armie Hammer Justin Theroux Mim...
272,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Documentary,destroyer,Nicole Kidman Sebastian Stan Toby Kebbell Kary...


## Extracting features of 2019 movies from Wikipedia

In [24]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [25]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [26]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
240,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
241,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[224]
242,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[225]
243,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[226]


In [27]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [28]:
df_2019 = df[['Title','Cast and crew','genres']]
df_2019

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama
...,...,...,...
240,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family
241,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance History
242,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama History
243,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History


In [29]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))

In [30]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))

In [31]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))

In [32]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [33]:
df_2019 = df_2019.rename(columns={'Title':'movie_title'})

In [34]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [35]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

In [36]:
new_df19['movie_title'] = new_df19['movie_title'].str.lower()

In [37]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']

In [38]:
new_df19

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,escape room,Taylor Russell Logan Miller Deborah Ann Woll A...
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama Action Crime,rust creek,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,american hangman,Donald Sutherland Vincent Kartheiser Oliver De...
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,a dog's way home,Bryce Dallas Howard Edward James Olmos Alexand...
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,the upside,Bryan Cranston Kevin Hart Nicole Kidman Neil B...
...,...,...,...,...,...,...,...
240,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
241,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
242,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama History,1917,George MacKay Dean-Charles Chapman Mark Strong...
243,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


In [39]:
my_df = new_df18.append(new_df19,ignore_index=True)

In [40]:
my_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama Western Thriller,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
514,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Tom Holland Rashida Jones Nick Brun...
515,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
516,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama History,1917,George MacKay Dean-Charles Chapman Mark Strong...
517,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...


## Extracting features of 2020 movies from Wikipedia

In [46]:
import bs4 as bs
import urllib.request

In [47]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [48]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [49]:
tables = soup.find_all('table',class_='wikitable sortable')

In [50]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [51]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [52]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
270,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
271,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
272,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
273,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [53]:
df_2020 = df[['Title','Cast and crew']]

In [54]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [56]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN


In [57]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [58]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [59]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [60]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [61]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [62]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [63]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [64]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [65]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [66]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes
271,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure Action,News of the World
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [67]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [68]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [70]:
my_df1 = my_df.append(new_df20,ignore_index=True)

In [71]:
my_df1

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama Western Thriller,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
789,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
790,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure Action,news of the world,NaN
791,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
792,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


## Extracting features of 2021 movies from Wikipedia

In [72]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [73]:
tables = soup.find_all('table',class_='wikitable sortable')

In [74]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [75]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [76]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
356,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[273]
357,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[274]
358,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[275]
359,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[276]


In [77]:
df_2021 = df[['Title','Cast and crew']]

In [78]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A..."
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...


In [80]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

In [81]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction
...,...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Mystery Science Fiction


In [82]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

In [83]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [84]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [85]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [86]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Mystery Science Fiction,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar


In [87]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [88]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [89]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire
...,...,...,...,...,...,...
356,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth
357,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan
358,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog
359,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Mystery Science Fiction,Memoria


In [90]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [91]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

In [92]:
my_df2 = my_df1.append(new_df21,ignore_index=True)

In [93]:
my_df2

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama Western Thriller,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
1150,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
1151,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
1152,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...
1153,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Mystery Science Fiction,memoria,Apichatpong Weerasethakul (director/acreenplay...


## Extracting features of 2022 movies from Wikipedia

In [94]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [95]:
tables = soup.find_all('table',class_='wikitable sortable')

In [96]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [97]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [98]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5],NaN
4,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
327,DECEMBER,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[263]
328,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
329,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN
330,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df_2022 = df[['Title','Cast and crew']]

In [101]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward Drake (director/screenplay); Timothy V....
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
328,NaN,NaN
329,NaN,NaN
330,NaN,NaN


In [102]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

In [103]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama
328,NaN,NaN,Comedy
329,NaN,NaN,Comedy
330,NaN,NaN,Comedy


In [104]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

In [105]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [106]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [107]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [108]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Family Animation Fantasy,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Edward Drake (director/screenplay); Timothy V....,Action Thriller Crime Drama,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
...,...,...,...,...,...,...,...
327,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick
328,NaN,NaN,Comedy,nan,nan,NaN,NaN
329,NaN,NaN,Comedy,nan,nan,NaN,NaN
330,NaN,NaN,Comedy,nan,nan,NaN,NaN


In [109]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [110]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [111]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward Drake,Timothy V. Murphy,Bruce Willis,NaN,Action Thriller Crime Drama,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
...,...,...,...,...,...,...
327,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama,"Alice, Darling"
328,nan,nan,NaN,NaN,Comedy,NaN
329,nan,nan,NaN,NaN,Comedy,NaN
330,nan,nan,NaN,NaN,Comedy,NaN


In [112]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [113]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

In [114]:
my_df3 = my_df2.append(new_df22,ignore_index=True)

In [115]:
my_df3

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama Western Thriller,sweet country,Bryan Brown Sam Neill unknown Warwick Thornton...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
1482,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama,"alice, darling",Anna Kendrick Kaniehtiio Horn Charlie Carrick ...
1483,nan,nan,NaN,NaN,Comedy,NaN,NaN
1484,nan,nan,NaN,NaN,Comedy,NaN,NaN
1485,nan,nan,NaN,NaN,Comedy,NaN,NaN


In [116]:
my_df3.isna().sum()

director_name     0
actor_1_name      0
actor_2_name     22
actor_3_name     89
genres            5
movie_title       5
comb             93
dtype: int64

In [117]:
my_df3.dropna(how='any',inplace=True)

In [120]:
my_df3.to_csv('latest.csv', index=False)